In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances_argmin_min

In [ ]:
# Carregar os dados
data = pd.read_csv("../data_source/amostra_total.csv", sep=';')
data = data[["INDICE", "LATITUDE", "LONGITUDE", "LOGRADOURO", "NUMERO"]]

In [ ]:
# Definir os parâmetros
first_n_clusters = 42
min_points_per_cluster = 350
max_points_per_cluster = 450

In [ ]:
intervals = {
        'Muito Abaixo (menos de 100)': (0, 99),
        'Abaixo (100-349)': (100, 349),
        'Dentro da Média (350-450)': (350, 450),
        'Acima (451-800)': (451, 800),
        'Muito Acima (mais de 800)': (801, float('inf'))
    }

In [ ]:
# Clusterização inicial
kmeans = MiniBatchKMeans(n_clusters=first_n_clusters, random_state=8081)
data['LEITURISTA'] = kmeans.fit_predict(data[['LATITUDE', 'LONGITUDE']])

In [ ]:

# Função para plotar os clusters
def plot_clusters_sns(df, num_clusters, cluster_col, title, filename):
    plt.figure(figsize=(10, 6))
    palette = sns.color_palette("hsv", num_clusters)
    sns.scatterplot(data=df, x='LONGITUDE', y='LATITUDE', hue=cluster_col, palette=palette, s=50, legend=None)
    plt.title(title)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.savefig(filename)
    plt.close()

In [ ]:
# Função para categorizar a quantidade de pontos em cada intervalo
def categorize_points(count):
    intervals = {
        'Muito Abaixo (menos de 100)': (0, 99),
        'Abaixo (100-349)': (100, 349),
        'Dentro da Média (350-450)': (350, 450),
        'Acima (451-800)': (451, 800),
        'Muito Acima (mais de 800)': (801, float('inf'))
    }
    for category, (low, high) in intervals.items():
        if low <= count <= high:
            return category
    return None


In [ ]:
# Função para rebalancear os clusters mantendo a proximidade geográfica
def rebalance_clusters(df, min_count, max_count):
    cluster_counts = df['LEITURISTA'].value_counts().to_dict()
    
    while True:
        # Identificar clusters desbalanceados
        clusters_above = {k: v for k, v in cluster_counts.items() if v > max_count}
        clusters_below = {k: v for k, v in cluster_counts.items() if v < min_count}
        
        if not clusters_above or not clusters_below:
            break
        
        # Selecionar clusters desbalanceados
        cluster_above = max(clusters_above, key=clusters_above.get)
        cluster_below = min(clusters_below, key=clusters_below.get)
        
        # Calcular número de pontos para transferir
        points_to_transfer = min(
            clusters_above[cluster_above] - max_count,
            min_count - clusters_below[cluster_below]
        )
        
        # Encontrar pontos a serem transferidos
        points_above = df[df['LEITURISTA'] == cluster_above]
        centroid_below = kmeans.cluster_centers_[cluster_below]
        _, indices = pairwise_distances_argmin_min(points_above[['LATITUDE', 'LONGITUDE']], [centroid_below])
        points_selected = points_above.iloc[indices[:points_to_transfer]]
        
        # Transferir pontos
        df.loc[points_selected.index, 'LEITURISTA'] = cluster_below
        cluster_counts[cluster_above] -= points_to_transfer
        cluster_counts[cluster_below] += points_to_transfer


In [ ]:
# Rebalancear os clusters
rebalance_clusters(data, min_points_per_cluster, max_points_per_cluster)

In [ ]:
# Recalcular os dias para os clusters ajustados
for leiturista in range(first_n_clusters):
    subcluster_data = data[data['LEITURISTA'] == leiturista]
    if len(subcluster_data) >= 22:
        kmeans_22 = MiniBatchKMeans(n_clusters=22, random_state=8081)
        data.loc[subcluster_data.index, 'DIA'] = kmeans_22.fit_predict(subcluster_data[['LATITUDE', 'LONGITUDE']])
    else:
        data.loc[subcluster_data.index, 'DIA'] = np.arange(len(subcluster_data))

In [ ]:
# Calcular as estatísticas
stats = []
for leiturista in range(first_n_clusters):
    subcluster_data = data[data['LEITURISTA'] == leiturista]
    dias_counts = subcluster_data['DIA'].value_counts()
    for dia, count in dias_counts.items():
        stats.append({
            'LEITURISTA': leiturista,
            'DIA': dia,
            'COUNT': count,
            'CATEGORY': categorize_points(count)
        })

stats_df = pd.DataFrame(stats)

In [ ]:
# Contagem de dias dentro de cada intervalo
category_counts = stats_df['CATEGORY'].value_counts().to_dict()

In [ ]:
# Adicionar categorias que não estão presentes no DataFrame
for category in intervals.keys():
    if category not in category_counts:
        category_counts[category] = 0

In [ ]:
print("Contagem de dias dentro de cada intervalo após redistribuição:")
for category, count in category_counts.items():
    print(f"{category}: {count}")

In [ ]:
# Salvar as novas estatísticas em um arquivo CSV
stats_df.to_csv(f'../cluster/{first_n_clusters}_clusters_stats_adjusted_with_categories.csv', index=False)

# Plotar os clusters ajustados
plot_clusters_sns(data, first_n_clusters, 'LEITURISTA', f'{first_n_clusters} Clusters Ajustados', f'../cluster/{first_n_clusters}_clusters_ajustados.png')